In [1]:
import os
from dotenv import load_dotenv
load_dotenv(encoding='utf-8')

True

# RAG pipeline

## Vector Store and Retriever

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from pymongo import MongoClient

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL")) # DEFAULT_OPENAI_MODEL='gpt-4o-mini-2024-07-18'

# embedding_model=OpenAIEmbeddings(model=os.getenv("DEFAULT_OPENAI_EMBEDDING"), disallowed_special=())
embedding_model=OpenAIEmbeddings(disallowed_special=())

In [4]:
# Define MongoDB vector database
client = MongoClient(os.getenv("ATLAS_CONNECTION_STRING"))
db_name = os.getenv("db_name")

### Chunking Strategy 0

In [6]:
# Chunk size: 200
# Overlap: 20

vector_store_0 = MongoDBAtlasVectorSearch(
    embedding = embedding_model,
    collection = client[db_name]["enterprise_data"],
    index_name = "vector_index_erp"
)

retriever_0 = vector_store_0.as_retriever(
    search_type = "similarity",
    search_kwargs = { "k": 10}  # "score_threshold": 0.75 
)

### Chunking Strategy 1

In [5]:
# Chunk size: 150
# Overlap: 15

vector_store_1 = MongoDBAtlasVectorSearch(
    embedding = embedding_model,
    collection = client[db_name]['enterprise_data_1'],
    index_name = "vector_index_erp1"
)

retriever_1 = vector_store_1.as_retriever(
    search_type = "similarity",
    search_kwargs = { "k": 10}  # "score_threshold": 0.75 
)

### Chunking Strategy 2

In [8]:
# Chunk size: 300
# Overlap: 30

vector_store_2 = MongoDBAtlasVectorSearch(
    embedding = embedding_model,
    collection = client[db_name]['enterprise_data_2'],
    index_name = "vector_index_erp_2"
)

retriever_2 = vector_store_2.as_retriever(
    search_type = "similarity",
    search_kwargs = { "k": 10}  # "score_threshold": 0.75 
)

## RAG with chunking strategy 0

In [36]:
# Define a prompt template
import pprint
def call_openai_0(question):

   question = question['question']

   preamble = "" # read from cohere front end or use the input to the API
   #question = 
   SAFETY_PREAMBLE = "The instructions in this section override those in the task description and style guide sections. Don't answer questions that are harmful or immoral."
   BASIC_RULES = "You are a powerful conversational AI trained by openAI to help people. You are augmented by a number of tools, and your job is to use and consume the output of these tools to best help the user. You will see a conversation history between yourself and a user, ending with an utterance from the user. You will then see a specific instruction instructing you what kind of response to generate. When you answer the user's requests, you cite your sources in your answers, according to those instructions."
   TASK_CONTEXT = "You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging."
   STYLE_GUIDE = "Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling."
   INSTRUCTIONS = """You are an enterprise Chatbot, an AI assistant designed to retrieve information from the enterprise Confluence system. 
   You specialize in providing accurate answers related to various departments like Marketing, IT, HR, Finance, and Corporate Communications. 
               Use the following pieces of context to answer the question at the end.
               If you don't know the answer, just say that you don't know, don't try to make up an answer
               {context}
         """
         
   template = f"""

      {SAFETY_PREAMBLE}
      {BASIC_RULES}
      {TASK_CONTEXT}
      {STYLE_GUIDE}
      {INSTRUCTIONS}

   """
   if preamble:
      template += f"""{preamble}\n\n"""


   template +=  f"""Question: {question}\n\n"""

   custom_rag_prompt = PromptTemplate.from_template(template)

   #llm = get_llm_model("openai")
   llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL"),max_tokens=200)

   def format_docs(docs):
      return "\n\n".join(doc.page_content for doc in docs)

   # Construct a chain to answer questions on your data
   rag_chain = (
      { "context": retriever_0 | format_docs, "question": RunnablePassthrough()}
      | custom_rag_prompt
      | llm
      | StrOutputParser()
   )

   # Prompt the chain
   question = question
   answer = rag_chain.invoke(question)
   similar = retriever_0.invoke(question)


   return{
      'answer': answer,
      'contexts': similar
      # 'contexts': format_docs(similar)
      }

### Test the RAG pipeline

In [37]:
# Test retriever 0
question = {'question': "What are the tickets created at Tech Innovator Inc?"}
answer = call_openai_0(question)

In [38]:
print(answer['answer'][:150])
answer['contexts']

The ticket created at Tech Innovators Inc. is as follows:

- **Ticket ID:** TI-67890
- **Created Date:** July 20, 2024
- **Priority:** High
- **Status


[Document(metadata={'_id': '66d8164c0533e009aa2d4c3e', 'pageid': '688130', 'department': 'Marketing', 'title': 'Marketing Campaign Results'}, page_content='of times Tech Innovators Inc. was mentioned across social media platforms.2. Lead GenerationLeads Generated: 15,000Description: Total number of leads captured through landing pages and sign-up'),
 Document(metadata={'_id': '66d816680533e009aa2d4e63', 'pageid': '688130', 'department': 'Marketing', 'title': 'Marketing Campaign Results'}, page_content='of times Tech Innovators Inc. was mentioned across social media platforms.2. Lead GenerationLeads Generated: 15,000Description: Total number of leads captured through landing pages and sign-up'),
 Document(metadata={'_id': '66d816300533e009aa2d4a19', 'pageid': '688130', 'department': 'Marketing', 'title': 'Marketing Campaign Results'}, page_content='of times Tech Innovators Inc. was mentioned across social media platforms.2. Lead GenerationLeads Generated: 15,000Description: Total number

In [40]:
answer['contexts'][0].page_content

'of times Tech Innovators Inc. was mentioned across social media platforms.2. Lead GenerationLeads Generated: 15,000Description: Total number of leads captured through landing pages and sign-up'

## RAG with chunking strategy 1

In [6]:
# Define a prompt template
import pprint
def call_openai_1(question):

   question = question['question']

   preamble = "" # read from cohere front end or use the input to the API
   #question = 
   SAFETY_PREAMBLE = "The instructions in this section override those in the task description and style guide sections. Don't answer questions that are harmful or immoral."
   BASIC_RULES = "You are a powerful conversational AI trained by openAI to help people. You are augmented by a number of tools, and your job is to use and consume the output of these tools to best help the user. You will see a conversation history between yourself and a user, ending with an utterance from the user. You will then see a specific instruction instructing you what kind of response to generate. When you answer the user's requests, you cite your sources in your answers, according to those instructions."
   TASK_CONTEXT = "You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging."
   STYLE_GUIDE = "Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling."
   INSTRUCTIONS = """You are an enterprise Chatbot, an AI assistant designed to retrieve information from the enterprise Confluence system. 
   You specialize in providing accurate answers related to various departments like Marketing, IT, HR, Finance, and Corporate Communications. 
               Use the following pieces of context to answer the question at the end.
               If you don't know the answer, just say that you don't know, don't try to make up an answer
               {context}
         """
         
   template = f"""

      {SAFETY_PREAMBLE}
      {BASIC_RULES}
      {TASK_CONTEXT}
      {STYLE_GUIDE}
      {INSTRUCTIONS}

   """
   if preamble:
      template += f"""{preamble}\n\n"""


   template +=  f"""Question: {question}\n\n"""

   custom_rag_prompt = PromptTemplate.from_template(template)

   #llm = get_llm_model("openai")
   llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL"),max_tokens=200)

   def format_docs(docs):
      return "\n\n".join(doc.page_content for doc in docs)

   # Construct a chain to answer questions on your data
   rag_chain = (
      { "context": retriever_1 | format_docs, "question": RunnablePassthrough()}
      | custom_rag_prompt
      | llm
      | StrOutputParser()
   )

   # Prompt the chain
   question = question
   answer = rag_chain.invoke(question)
   similar = retriever_1.invoke(question)


   return{
      'answer': answer,
      'contexts': similar
      }

In [7]:
# Test retriever 1
question = {'question': "What are the tickets created at Tech Innovator Inc?"}
answer = call_openai_1(question)

In [8]:
print(answer['answer'][:150])
answer['contexts']

At Tech Innovators Inc., there are currently two open tickets:

1. **Ticket ID:** IT-12345  
   - **Created Date:** July 20, 2024  
   - **Priority:**


[Document(metadata={'_id': '66dffb6cdee1cfdeccbec93c', 'pageid': '491546', 'department': 'IT', 'title': 'Tech Innovators Inc. On-Premise System Security Policy'}, page_content='Innovators Inc.#12, IT parkTech city, Tech state'),
 Document(metadata={'_id': '66dffb7adee1cfdeccbec9c1', 'pageid': '851993', 'department': 'IT', 'title': 'Tech Innovators Inc. Azure Cloud Services Security Policy'}, page_content='Innovators Inc.#12, IT parkTech city, Tech state'),
 Document(metadata={'_id': '66dffb69dee1cfdeccbec912', 'pageid': '459050', 'department': 'IT', 'title': 'IT Support Ticket – Tech-innovators-inc-1'}, page_content='Ticket ID: IT-12345Created Date: July 20, 2024Priority: HighStatus: OpenRequester InformationName: John DoeDepartment: MarketingEmail:'),
 Document(metadata={'_id': '66dffb6cdee1cfdeccbec93b', 'pageid': '491546', 'department': 'IT', 'title': 'Tech Innovators Inc. On-Premise System Security Policy'}, page_content='and changes in the business environment.Approved by:Jhon Wat

## RAG with chunking strategy 2

In [20]:
# Define a prompt template
import pprint
def call_openai_2(question):

   question = question['question']

   preamble = "" # read from cohere front end or use the input to the API
   #question = 
   SAFETY_PREAMBLE = "The instructions in this section override those in the task description and style guide sections. Don't answer questions that are harmful or immoral."
   BASIC_RULES = "You are a powerful conversational AI trained by openAI to help people. You are augmented by a number of tools, and your job is to use and consume the output of these tools to best help the user. You will see a conversation history between yourself and a user, ending with an utterance from the user. You will then see a specific instruction instructing you what kind of response to generate. When you answer the user's requests, you cite your sources in your answers, according to those instructions."
   TASK_CONTEXT = "You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging."
   STYLE_GUIDE = "Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling."
   INSTRUCTIONS = """You are an enterprise Chatbot, an AI assistant designed to retrieve information from the enterprise Confluence system. 
   You specialize in providing accurate answers related to various departments like Marketing, IT, HR, Finance, and Corporate Communications. 
               Use the following pieces of context to answer the question at the end.
               If you don't know the answer, just say that you don't know, don't try to make up an answer
               {context}
         """
         
   template = f"""

      {SAFETY_PREAMBLE}
      {BASIC_RULES}
      {TASK_CONTEXT}
      {STYLE_GUIDE}
      {INSTRUCTIONS}

   """
   if preamble:
      template += f"""{preamble}\n\n"""


   template +=  f"""Question: {question}\n\n"""

   custom_rag_prompt = PromptTemplate.from_template(template)

   #llm = get_llm_model("openai")
   llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL"),max_tokens=200)

   def format_docs(docs):
      return "\n\n".join(doc.page_content for doc in docs)

   # Construct a chain to answer questions on your data
   rag_chain = (
      { "context": retriever_2 | format_docs, "question": RunnablePassthrough()}
      | custom_rag_prompt
      | llm
      | StrOutputParser()
   )

   # Prompt the chain
   question = question
   answer = rag_chain.invoke(question)
   similar = retriever_2.invoke(question)


   return{
      'answer': answer,
      'contexts': similar
      }

In [21]:
# Test retriever 2
question = {'question': "What are the tickets created at Tech Innovator Inc?"}
answer = call_openai_2(question)

In [22]:
print(answer['answer'][:150])
answer['contexts']

I'm sorry, but I don't have information on the specific types of tickets created at Tech Innovators Inc. If you have a particular context or departmen


["page_content='individuals with disabilities, we strive to build a dynamic and innovative team. Continuous improvement and adherence to ethical standards are at the core of our recruitment practices, ensuring Tech Innovators Inc. remains a leader in the industry.' metadata={'_id': '66eb06d8378ec16fc5706113', 'pageid': '655683', 'department': 'HR', 'title': 'Tech Innovators Inc. Recruitment Policies Guidelines'}",
 "page_content='individuals with disabilities, we strive to build a dynamic and innovative team. Continuous improvement and adherence to ethical standards are at the core of our recruitment practices, ensuring Tech Innovators Inc. remains a leader in the industry.' metadata={'_id': '66eb0672378ec16fc5705ffd', 'pageid': '655683', 'department': 'HR', 'title': 'Tech Innovators Inc. Recruitment Policies Guidelines'}",
 "page_content='code policy?Training and DevelopmentQ1: How does Tech Innovators Inc. support employee training and development?A1: Tech Innovators Inc. is committe

# RAG pipeline evaluation

## Test data set prep

In [9]:
import pandas as pd
import json

def json_to_dataframe(json_file_path):
  """Reads a JSON file and converts it to a pandas DataFrame.

  Args:
    json_file_path (str): The path to the JSON file.

  Returns:
    pandas.DataFrame: The DataFrame created from the JSON data.
  """

  with open(json_file_path, 'r') as f:
    data = json.load(f)

  # Handle different JSON structures
  if isinstance(data, list):
    # If the JSON data is a list of dictionaries, create a DataFrame directly
    df = pd.DataFrame(data)
  elif isinstance(data, dict):
    # If the JSON data is a single dictionary, convert it to a list of dictionaries
    df = pd.DataFrame([data])
  else:
    raise ValueError("Unsupported JSON structure")

  return df

In [10]:
# Example usage:
from from_root import from_root
import os
file_name = "test_dataset_it.json"
json_file_path = os.path.join(from_root(), "data-test/test_dataset/", file_name)
data_to_test = json_to_dataframe(json_file_path)

## RAGAS evaluation

In [21]:
import json
import pandas as pd

def serialize_list(value):
    """Serializes a list to a JSON string."""
    return json.dumps(value)

def deserialize_list(value):
    """Deserializes a JSON string back into a list."""
    return json.loads(value)

In [22]:
def extract_page_content(documents):
    return [doc.page_content for doc in documents]

### Chunking Strategy 0

In [44]:
# Generate all the answers for the questions in the dataset
# examples = client.list_examples(dataset_name="hr test")
answers_0 = []
contexts_0 = []
# retrieved_contexts = []
for question in data_to_test['question']:
    question_dict = {'question': question}
    answer = call_openai_0(question_dict)
    contexts_0.append(answer['contexts'])
    answers_0.append(answer['answer'])

In [45]:
# update the dataset with answers and contexts
data_to_test['answers'] = answers_0
data_to_test['contexts'] = contexts_0

In [ ]:
# Replace empty list context with ['No context'] if there are any
def is_empty_list(lst):
    return len(lst) == 0
data_to_test['contexts'] = data_to_test['contexts'].apply(lambda x: ['No context'] if is_empty_list(x) else x)

In [56]:
from datasets import Dataset

question = list(data_to_test['question'])
answer = list(data_to_test['answers'])
contexts = list(data_to_test['contexts'].apply(extract_page_content))
ground_truth = list(data_to_test['ground_truth'])

data_samples_0 = {
    'question': question,
    'answer': answer,
    'contexts': contexts,
    'ground_truth': ground_truth
}

dataset_0 = Dataset.from_dict(data_samples_0)

In [ ]:
# Optional, uncomment to trace runs with LangSmith. Sign up here: https://smith.langchain.com.
# from langsmith import Client
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_PROJECT"] = "Cohere_RAG_Online_Eval"
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
# client = Client()

In [ ]:
from ragas import evaluate
# from ragas.integrations.langsmith import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
result_0 = evaluate(
    dataset_0,
    metrics=[
        answer_relevancy,
        faithfulness,
        context_recall,
        context_precision,
    ],
)

In [59]:
result_0

{'answer_relevancy': 0.9552, 'faithfulness': 0.4109, 'context_recall': 0.3813, 'context_precision': 0.6624}

In [60]:
result_0.to_pandas()

,question,answer,contexts,ground_truth,answer_relevancy,faithfulness,context_recall,context_precision
0,How does the role of the Senior Director respo...,The Senior Director responsible for Analytics ...,[the Senior Director responsible for Analytics...,The role of the Senior Director responsible fo...,0.925206,0.307692,1.000000,1.000000
1,What is the importance of identifying and addr...,Identifying and addressing growth areas in sel...,[to identify strengths and areas for improveme...,Identifying and addressing growth areas in sel...,0.990183,0.187500,0.333333,0.000000
2,What forms of unethical behavior are strictly ...,"In the recruitment process of Inc., any form o...",[Inc. upholds the highest ethical standards in...,Favoritism or nepotism,0.920672,0.875000,0.000000,0.961735
3,What is the significance of emotional and aest...,Emotional and aesthetic labor plays a signific...,[LabourEmotional and aesthetic labor involves ...,Emotional and aesthetic labor in the workplace...,0.974615,0.565217,0.666667,1.000000
4,What is the purpose of the orientation session...,The purpose of the orientation session at Tech...,[see you thrive at Tech Innovators Inc. Welcom...,The purpose of the orientation session at Tech...,1.000000,0.222222,0.000000,0.000000
5,What mechanisms are in place for reporting vio...,"At Tech Innovators Inc., employees can report ...",[and identify areas for improvement.5.3 Report...,Employees can report violations of labor laws ...,0.962563,0.400000,0.250000,1.000000
6,How do employee engagement and disengagement d...,Employee engagement and disengagement differ s...,"[are motivated and committed, disengaged emplo...",Employee engagement and disengagement differ i...,0.920337,0.529412,1.000000,1.000000
7,What steps are needed to extract data from Con...,To extract data from Confluence and create a R...,[IntroductionThis guide provides a step-by-ste...,To extract data from Confluence and create a R...,0.921967,0.571429,0.181818,1.000000
8,How does Tech Innovators Inc. promote employee...,Tech Innovators Inc. promotes employee engagem...,[IntroductionTech Innovators Inc. is committed...,Tech Innovators Inc. promotes employee engagem...,0.980843,0.040000,0.000000,0.000000


In [62]:
file_name = "test_dataset_it_chunking_0.csv"
json_file_path = os.path.join(from_root(), "data-test/test_dataset/", file_name)
result_0.to_pandas().to_csv(json_file_path, index=False)

### Chunking Strategy 1

In [12]:
from from_root import from_root
import os
file_name = "test_dataset_it.json"
json_file_path = os.path.join(from_root(), "data-test/test_dataset/", file_name)
data_to_test_1 = json_to_dataframe(json_file_path)

In [13]:
# Generate all the answers for the questions in the dataset
# examples = client.list_examples(dataset_name="hr test")
answers_1 = []
contexts_1 = []
# retrieved_contexts = []
for question in data_to_test_1['question']:
    question_dict = {'question': question}
    answer = call_openai_1(question_dict)
    contexts_1.append(answer['contexts'])
    answers_1.append(answer['answer'])

In [14]:
# update the dataset with answers
data_to_test_1['answers'] = answers_1
data_to_test_1['contexts'] = contexts_1

In [15]:
# Replace empty list context with ['No context'] if there are any
def is_empty_list(lst):
    return len(lst) == 0
data_to_test_1['contexts'] = data_to_test_1['contexts'].apply(lambda x: ['No context'] if is_empty_list(x) else x)

In [23]:
from datasets import Dataset

question = list(data_to_test_1['question'])
answer = list(data_to_test_1['answers'])
contexts = list(data_to_test_1['contexts'].apply(extract_page_content))
ground_truth = list(data_to_test_1['ground_truth'])

data_samples_1 = {
    'question': question,
    'answer': answer,
    'contexts': contexts,
    'ground_truth': ground_truth
}

dataset_1 = Dataset.from_dict(data_samples_1)

In [ ]:
# Optional, uncomment to trace runs with LangSmith. Sign up here: https://smith.langchain.com.
# from langsmith import Client
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_PROJECT"] = "Cohere_RAG_Online_Eval"
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
# client = Client()

In [26]:
from ragas import evaluate
# from ragas.integrations.langsmith import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
result_1 = evaluate(
    dataset_1,
    metrics=[
        answer_relevancy,
        faithfulness,
        context_recall,
        context_precision,
    ],
)

Evaluating:   0%|          | 0/36 [00:00<?, ?it/s]

In [27]:
result_1

{'answer_relevancy': 0.6372, 'faithfulness': 0.2407, 'context_recall': 0.0000, 'context_precision': 0.0000}

In [28]:
result_1.to_pandas()

,question,answer,contexts,ground_truth,answer_relevancy,faithfulness,context_recall,context_precision
0,How does the role of the Senior Director respo...,The Senior Director responsible for Analytics ...,[immediate attention to avoid delays in custom...,The role of the Senior Director responsible fo...,0.983052,0.000000,0.0,0.0
1,What is the importance of identifying and addr...,Identifying and addressing growth areas in sel...,[must receive regular training on Azure securi...,Identifying and addressing growth areas in sel...,0.908362,0.000000,0.0,0.0
2,What forms of unethical behavior are strictly ...,The recruitment process at Tech Innovators Inc...,[promptly.9.2 PenaltiesEmployees found in viol...,Favoritism or nepotism,0.934730,0.000000,0.0,0.0
3,What is the significance of emotional and aest...,Emotional and aesthetic labor in the workplace...,[promptly.9.2 PenaltiesEmployees found in viol...,Emotional and aesthetic labor in the workplace...,0.979084,0.000000,0.0,0.0
4,What is the purpose of the orientation session...,"I'm sorry, but I don't have information regard...","[Innovators Inc.#12, IT parkTech city, Tech st...",The purpose of the orientation session at Tech...,0.000000,1.000000,0.0,0.0
5,What mechanisms are in place for reporting vio...,I don't have specific information regarding th...,[automated tools.Any violations of this policy...,Employees can report violations of labor laws ...,0.000000,0.333333,0.0,0.0
6,How do employee engagement and disengagement d...,Employee engagement and disengagement differ s...,[promptly.9.2 PenaltiesEmployees found in viol...,Employee engagement and disengagement differ i...,0.933077,0.000000,0.0,0.0
7,What steps are needed to extract data from Con...,To extract data from Confluence and create a R...,[com.atlassian.confluence.project-linker\nproj...,To extract data from Confluence and create a R...,0.996660,0.500000,0.0,0.0
8,How does Tech Innovators Inc. promote employee...,"I'm sorry, but I don't have information on how...","[Innovators Inc.#12, IT parkTech city, Tech st...",Tech Innovators Inc. promotes employee engagem...,0.000000,0.333333,0.0,0.0


In [29]:
# Save the extracted data for evaluation use
file_name = "eval_result_dataset_hr_chunking1.csv"
result_1.to_pandas().to_csv(os.path.join(from_root(), "data-test/test_dataset/", file_name), index=False)

### Chunking Strategy 2

In [79]:
from from_root import from_root
import os
file_name = "test_dataset_it.json"
json_file_path = os.path.join(from_root(), "data-test/test_dataset/", file_name)
data_to_test_2 = json_to_dataframe(json_file_path)

In [81]:
# Generate all the answers for the questions in the dataset
# examples = client.list_examples(dataset_name="hr test")
answers_2 = []
contexts_2 = []
# retrieved_contexts = []
for question in data_to_test_2['question']:
    question_dict = {'question': question}
    answer = call_openai_2(question_dict)
    contexts_2.append(answer['contexts'])
    answers_2.append(answer['answer'])

In [83]:
# update the dataset with answers and contexts
data_to_test_2['answers'] = answers_2
data_to_test_2['contexts'] = contexts_2

In [ ]:
# Replace empty list context with ['No context'] if there are any 
def is_empty_list(lst):
    return len(lst) == 0
data_to_test_2['contexts'] = data_to_test_2['contexts'].apply(lambda x: ['No context'] if is_empty_list(x) else x)

In [88]:
from datasets import Dataset

question = list(data_to_test_2['question'])
answer = list(data_to_test_2['answers'])
contexts = list(data_to_test_2['contexts'])
ground_truth = list(data_to_test_2['ground_truth'])

data_samples_2 = {
    'question': question,
    'answer': answer,
    'contexts': contexts,
    'ground_truth': ground_truth
}

dataset_2 = Dataset.from_dict(data_samples_2)

In [ ]:
# Optional, uncomment to trace runs with LangSmith. Sign up here: https://smith.langchain.com.
# from langsmith import Client
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_PROJECT"] = "Cohere_RAG_Online_Eval"
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
# client = Client()

In [90]:
from ragas import evaluate
# from ragas.integrations.langsmith import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
result_2 = evaluate(
    dataset_2,
    metrics=[
        answer_relevancy,
        faithfulness,
        context_recall,
        context_precision,
    ],
)

Evaluating:   0%|          | 0/36 [00:00<?, ?it/s]

In [91]:
result_2

{'answer_relevancy': 0.9585, 'faithfulness': 0.5833, 'context_recall': 0.5242, 'context_precision': 0.6198}

In [92]:
result_2.to_pandas()

,question,answer,contexts,ground_truth,answer_relevancy,faithfulness,context_recall,context_precision
0,How does the role of the Senior Director respo...,The Senior Director responsible for Analytics ...,[page_content='initiatives.Feedback Loop: Impl...,The role of the Senior Director responsible fo...,0.921495,0.500000,0.545455,0.785670
1,What is the importance of identifying and addr...,Identifying and addressing growth areas in sel...,[page_content='assessments are used to ensure ...,Identifying and addressing growth areas in sel...,0.990183,0.400000,0.000000,0.000000
2,What forms of unethical behavior are strictly ...,In the recruitment process at Tech Innovators ...,[page_content='standards in all recruitment ac...,Favoritism or nepotism,0.886105,0.857143,1.000000,0.683333
3,What is the significance of emotional and aest...,Emotional and aesthetic labor are significant ...,[page_content='healthy work-life balance and p...,Emotional and aesthetic labor in the workplace...,0.976493,0.750000,1.000000,1.000000
4,What is the purpose of the orientation session...,The purpose of the orientation session at Tech...,[page_content='Welcome to Tech Innovators Inc....,The purpose of the orientation session at Tech...,0.999999,0.181818,0.000000,0.250000
5,What mechanisms are in place for reporting vio...,"At Tech Innovators Inc., employees can report ...",[page_content='IntroductionTech Innovators Inc...,Employees can report violations of labor laws ...,0.962563,0.666667,0.500000,0.281548
6,How do employee engagement and disengagement d...,Employee engagement and disengagement differ s...,[page_content='Employee Engagement?Employee en...,Employee engagement and disengagement differ i...,0.960525,0.782609,1.000000,0.864435
7,What steps are needed to extract data from Con...,To extract data from Confluence and create a R...,[page_content='IntroductionThis guide provides...,To extract data from Confluence and create a R...,0.948293,0.666667,0.272727,0.880258
8,How does Tech Innovators Inc. promote employee...,Tech Innovators Inc. promotes employee engagem...,[page_content='opportunities. Employment Relat...,Tech Innovators Inc. promotes employee engagem...,0.980843,0.444444,0.400000,0.833333


In [93]:
# Save the extracted data for evaluation use
file_name = "eval_result_dataset_hr_chunking2.csv"
result_2.to_pandas().to_csv(os.path.join(from_root(), "data-test/test_dataset/", file_name), index=False)